In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# preprocessing: artificially increase dataset size
# addition of rotated and scaled versions of original images
# generated on the fly during training

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [3]:
img = load_img("data/train/cats/cat000.jpg")
x = img_to_array(img)
print(x.shape)

(374, 500, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [5]:
# Model #1

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3))) # changed column order
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [6]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
model.fit(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=2000 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
125/125 [==============================] - 25s 199ms/step - loss: 0.7073 - accuracy: 0.5275 - val_loss: 0.6708 - val_accuracy: 0.5835
Epoch 2/50
125/125 [==============================] - 25s 200ms/step - loss: 0.6648 - accuracy: 0.6145 - val_loss: 0.6565 - val_accuracy: 0.6215
Epoch 3/50
125/125 [==============================] - 26s 204ms/step - loss: 0.6381 - accuracy: 0.6370 - val_loss: 0.6027 - val_accuracy: 0.6810
Epoch 4/50
125/125 [==============================] - 25s 201ms/step - loss: 0.6081 - accuracy: 0.6780 - val_loss: 0.5805 - val_accuracy: 0.7010
Epoch 5/50
125/125 [==============================] - 25s 200ms/step - loss: 0.5840 - accuracy: 0.6965 - val_loss: 0.5982 - val_accuracy: 0.6760
Epoch 6/50
125/125 [==============================] - 25s 199ms/step - loss: 0.5721 - accuracy: 0.7175 - val_loss: 0.5676 - val_accuracy: 0.7085
Epoch 7/50
125/125 [==============================] - 25s 200ms/step - loss: 0.5621 - accuracy: 0.7150 - val_loss: 0.6056 - val_ac

In [8]:
# using pre-trained model (VGG16)
# https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
# get model up to last maxpool

from tensorflow.keras import applications

model = applications.VGG16(include_top=False)

In [9]:
batch_size = 16

generator = datagen.flow_from_directory(
        'data/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
bottleneck_features_train = model.predict(generator)
# save the output as a Numpy array
np.save('bottleneck_features_train.npy', bottleneck_features_train)

generator = datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = model.predict(generator)
np.save('bottleneck_features_validation.npy', bottleneck_features_validation)

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [10]:
train_data = np.load('bottleneck_features_train.npy')
print(train_data.shape)
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load('bottleneck_features_validation.npy')
print(validation_data.shape)
validation_labels = np.array([0] * 1000 + [1] * 1000)

print(train_data.shape)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')

(2000, 4, 4, 512)
(2000, 4, 4, 512)
(2000, 4, 4, 512)
Epoch 1/50
125/125 [==============================] - 1s 10ms/step - loss: 0.8592 - accuracy: 0.6560 - val_loss: 0.4550 - val_accuracy: 0.7720
Epoch 2/50
125/125 [==============================] - 1s 9ms/step - loss: 0.5204 - accuracy: 0.7610 - val_loss: 0.5239 - val_accuracy: 0.7735
Epoch 3/50
125/125 [==============================] - 1s 9ms/step - loss: 0.4571 - accuracy: 0.7965 - val_loss: 0.3924 - val_accuracy: 0.8215
Epoch 4/50
125/125 [==============================] - 1s 9ms/step - loss: 0.4011 - accuracy: 0.8260 - val_loss: 0.4563 - val_accuracy: 0.7960
Epoch 5/50
125/125 [==============================] - 1s 9ms/step - loss: 0.3523 - accuracy: 0.8530 - val_loss: 0.3797 - val_accuracy: 0.8345
Epoch 6/50
125/125 [==============================] - 1s 9ms/step - loss: 0.3086 - accuracy: 0.8710 - val_loss: 0.5573 - val_accuracy: 0.7925
Epoch 7/50
125/125 [==============================] - 1s 9ms/step - loss: 0.2970 - accuracy: 

In [11]:
# fine-tuning

from tensorflow.keras import optimizers

model = Sequential()
model.add(applications.VGG16(include_top=False))

print(len(model.layers))
print(model.output_shape)

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=(4,4,512)))#model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights('bottleneck_fc_model.h5')

# add the model on top of the convolutional base
model.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

1
(None, None, None, 512)


In [12]:
batch_size = 16

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

# fine-tune the model
model.fit(
        train_generator,
        steps_per_epoch=2000 // batch_size,
    
        epochs=50,
        validation_data=validation_generator,
        validation_steps=2000 // batch_size)

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 191s 2s/step - loss: 1.1621 - accuracy: 0.8795 - val_loss: 1.4631 - val_accuracy: 0.8655
Epoch 2/50
125/125 [==============================] - 191s 2s/step - loss: 1.0361 - accuracy: 0.8875 - val_loss: 1.4631 - val_accuracy: 0.8655
Epoch 3/50
125/125 [==============================] - 191s 2s/step - loss: 1.1412 - accuracy: 0.8730 - val_loss: 1.4631 - val_accuracy: 0.8655
Epoch 4/50
125/125 [==============================] - 191s 2s/step - loss: 1.1434 - accuracy: 0.8760 - val_loss: 1.4631 - val_accuracy: 0.8655
Epoch 5/50
125/125 [==============================] - 188s 2s/step - loss: 1.1226 - accuracy: 0.8895 - val_loss: 1.4631 - val_accuracy: 0.8655
Epoch 6/50
125/125 [==============================] - 185s 1s/step - loss: 1.0473 - accuracy: 0.8780 - val_loss: 1.4631 - val_accuracy: 0.8655
Epoch 7/50
125/125 [==============================] - 185s